In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [4]:
image_size = (200, 200)
batch_size = 128

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "images_in",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 500 files belonging to 5 classes.
Using 400 files for training.
Using 100 files for validation.


In [7]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [10]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [12]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [13]:
epochs = 25

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/25
4/4 [==============================] - 513s 111s/step - loss: 0.5494 - accuracy: 0.1850 - val_loss: 0.5476 - val_accuracy: 0.2000
Epoch 2/25
4/4 [==============================] - 505s 112s/step - loss: -2.5458 - accuracy: 0.2025 - val_loss: 0.3863 - val_accuracy: 0.2000
Epoch 3/25
4/4 [==============================] - 1671s 499s/step - loss: -4.8767 - accuracy: 0.1975 - val_loss: 0.2000 - val_accuracy: 0.2000
Epoch 4/25
4/4 [==============================] - 541s 120s/step - loss: -7.3633 - accuracy: 0.2000 - val_loss: -0.0128 - val_accuracy: 0.2000
Epoch 5/25
4/4 [==============================] - 551s 122s/step - loss: -9.2542 - accuracy: 0.1975 - val_loss: -0.2482 - val_accuracy: 0.2000
Epoch 6/25
4/4 [==============================] - 587s 136s/step - loss: -11.4506 - accuracy: 0.2000 - val_loss: -0.5107 - val_accuracy: 0.2000
Epoch 7/25
4/4 [==============================] - 574s 125s/step - loss: -13.5229 - accuracy: 0.2000 - val_loss: -0.8047 - val_accuracy: 0.2000


In [14]:
model.save('model_face_classifier.h5')

c:\Users\alish\Anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
